In [67]:
import time
import uuid
from datetime import datetime
from cassandra.cqlengine import columns
from cassandra.cqlengine.models import Model
from cassandra.cqlengine.management import sync_table


# ==============================================================================
class TestTable(Model):
    id = columns.UUID(primary_key=True, default=uuid.uuid4)
    text = columns.Text()
    number = columns.Integer()
    time_created = columns.DateTime(default=datetime.utcnow)


# ==============================================================================
class ChatGroup(Model):
    id = columns.UUID(primary_key=True, default=uuid.uuid4)
    name = columns.Text(0, 128)
    time_created = columns.DateTime(primary_key=True, clustering_order="DESC", default=datetime.utcnow)


# ==============================================================================
class ChatParticipantByAccount(Model):
    id = columns.UUID(default=uuid.uuid4)
    message_notify = columns.Boolean()

    id_account = columns.Integer(primary_key=True)
    id_chatgroup = columns.UUID(primary_key=True, clustering_order="DESC")


# ==============================================================================
class ChatParticipantByChatGroup(Model):
    id = columns.UUID(default=uuid.uuid4)
    message_notify = columns.Boolean()

    id_chatgroup = columns.UUID(primary_key=True)
    id_account = columns.Integer(primary_key=True, clustering_order="DESC")


# ==============================================================================
class ChatMessage(Model):
    id = columns.UUID(default=uuid.uuid4)
    text = columns.Text(0, 512)
    id_chatgroup = columns.UUID(primary_key=True)
    time_created = columns.DateTime(primary_key=True, clustering_order="DESC", default=datetime.utcnow)
    id_chatparticipant = columns.UUID(primary_key=True)


# ==============================================================================
def sync_tables():
    sync_table(TestTable)
    sync_table(ChatGroup)
    sync_table(ChatParticipantByAccount)
    sync_table(ChatParticipantByChatGroup)
    sync_table(ChatMessage)


In [68]:
from configurations.conf import Scylla
from cassandra.query import tuple_factory
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT, ConsistencyLevel
from cassandra.policies import WhiteListRoundRobinPolicy
from cassandra.cqlengine import connection

profile = ExecutionProfile(
    load_balancing_policy=WhiteListRoundRobinPolicy(['127.0.0.1']),
    consistency_level=ConsistencyLevel.LOCAL_QUORUM,
    serial_consistency_level=ConsistencyLevel.LOCAL_SERIAL,
    request_timeout=15,
    row_factory=tuple_factory
)
connection.setup(['127.0.0.1'], Scylla.DB_KEYSPACE,
                 execution_profiles={EXEC_PROFILE_DEFAULT: profile}, port=Scylla.DB_PORT)
connection.execute(f"DROP KEYSPACE {Scylla.DB_KEYSPACE}")
connection.execute(
    f"CREATE KEYSPACE IF NOT EXISTS {Scylla.DB_KEYSPACE} WITH replication = " +
    f"{{'class': 'NetworkTopologyStrategy', 'replication_factor': {Scylla.DB_REPLICATION_FACTOR}}}")
connection.execute(f"USE {Scylla.DB_KEYSPACE}")
sync_tables()

In [71]:
new_group = ChatGroup.create(name="temporary")
new_group

ChatGroup(id=UUID('5bf5b44c-c83a-41cc-a22a-6ca5b06f2ecc'), name='temporary', time_created=datetime.datetime(2023, 9, 1, 15, 52, 26, 9428))

In [72]:
new_group = ChatGroup.create(name="temporary 2")
new_group

ChatGroup(id=UUID('89e5f88c-637c-4b47-bebc-9c3982042276'), name='temporary 2', time_created=datetime.datetime(2023, 9, 1, 15, 52, 26, 526619))

In [54]:
new_participant_1 = ChatParticipant.create(message_notify=False, id_account=1,
                                           id_chatgroup='015edba8-22ea-4d4d-85d0-530c5400f9b8')

In [55]:
new_participant_2 = ChatParticipant.create(message_notify=False, id_account=2,
                                           id_chatgroup='015edba8-22ea-4d4d-85d0-530c5400f9b8')

In [56]:
new_participant_3 = ChatParticipant.create(message_notify=False, id_account=2,
                                           id_chatgroup='015edba8-22ea-4d4d-85d0-530c5400f9b8')

In [57]:
list_group_participant = ChatParticipant.objects.filter(id_account=2)

In [58]:
for i in list_group_participant.all():
    print(i)

ChatParticipant <id_account=2, id_chatgroup=015edba8-22ea-4d4d-85d0-530c5400f9b8>


In [40]:
import time, asyncio


def waitfunc(t: int, name):
    time.sleep(t)
    print(name)


async def asyncwaitfunc(t: int, name):
    await asyncio.sleep(t)
    print(name)


async def testconcurrent():
    await asyncwaitfunc(1, "hello")
    await asyncwaitfunc(2, "world")
    waitfunc(0.5, "test")


async def testconcurrent2():
    task1 = asyncio.create_task(asyncwaitfunc(1, "hello"))
    task2 = asyncio.create_task(asyncwaitfunc(2, "world"))
    await task1
    await task2
    waitfunc(0.5, "test")


In [41]:
await testconcurrent()

hello
world
test


In [42]:
await testconcurrent2()

hello
world
test
